In [1]:
import pandas as pd
import numpy as np

In [2]:
# getting movie data
movie_data=pd.read_csv('C:/Users/HP/Desktop/Movie_lens-Data/movies.csv',index_col=0)
movie_data.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
# Describing the data 
movie_data.describe()

,title,genres
count,9742,9742
unique,9737,951
top,Saturn 3 (1980),Drama
freq,2,1053


In [4]:
#geting the ratings data
movie_ratings=pd.read_csv('C:/Users/HP/Desktop/Movie_lens-Data/ratings.csv')
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# drop the column timestamp
movie_ratings.drop(["timestamp"],axis=1,inplace=True)
movie_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
movie_ratings.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


In [7]:
#shape of data
movie_data.shape, movie_ratings.shape

((9742, 2), (100836, 3))

In [8]:
# get the title of movie based on movie id
def movieTitle(movieId):
    title=movie_data.at[movieId,'title']
    return title
movieTitle(1)

'Toy Story (1995)'

In [9]:
# get the genre of the movie based on index
def movieGenre(movieId):
    genres=movie_data.at[movieId,'genres']
    return genres
movieGenre(1)

'Adventure|Animation|Children|Comedy|Fantasy'

In [10]:
# getting the N favorite movies of a particular user
def favMovie(userId,N):
    user=movie_ratings[movie_ratings.userId==userId] # gets all movie id an ratings of user
    
    # sorted the ratings in descending order and getting first N rows
    userMovies=pd.DataFrame.sort_values(user,['rating'],ascending=False)[:N]
    userMovies['title']=userMovies['movieId'].apply(movieTitle)
    userMovies['geners']=userMovies['movieId'].apply(movieGenre)
    return userMovies
favMovie(1,5)

,userId,movieId,rating,title,geners
231,1,5060,5.0,M*A*S*H (a.k.a. MASH) (1970),Comedy|Drama|War
185,1,2872,5.0,Excalibur (1981),Adventure|Fantasy
89,1,1291,5.0,Indiana Jones and the Last Crusade (1989),Action|Adventure
90,1,1298,5.0,Pink Floyd: The Wall (1982),Drama|Musical
190,1,2948,5.0,From Russia with Love (1963),Action|Adventure|Thriller


In [11]:
# getting total number of users on a particular movie
user_per_movie=movie_ratings.movieId.value_counts()
user_per_movie

356       329
318       317
296       307
593       279
2571      278
         ... 
5986        1
100304      1
34800       1
83976       1
8196        1
Name: movieId, Length: 9724, dtype: int64

In [12]:
# taking only those movies ratings which is seen by atleast 11 users
#movie_ratings=movie_ratings[movie_ratings.index.isin(user_per_movie[user_per_movie > 10].index)]
#movie_ratings.shape

In [13]:
# Making user movie matrix based on ratings given by user 
user_movie_matrix=pd.pivot_table(movie_ratings, index=['userId'],columns=['movieId'], values='rating')
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Find K Nearest Neighbours

In [14]:
#getting ratings of a particular user which he had watched
user1=100
user2=200

user1_ratings=user_movie_matrix.transpose()[user1]
user1_ratings.head()

movieId
1    NaN
2    NaN
3    3.5
4    NaN
5    NaN
Name: 100, dtype: float64

In [15]:
user2_ratings=user_movie_matrix.transpose()[user2]
user2_ratings.head()

movieId
1    3.5
2    NaN
3    NaN
4    NaN
5    4.0
Name: 200, dtype: float64

In [16]:
from scipy.spatial.distance import hamming
# hamming() returns a value which shows the percentage of disagreement

# Example
hamming(user1_ratings, user2_ratings)

0.9987659399424106

In [17]:
# function for the same
def distance(user1,user2):
    try:
        user1_ratings=user_movie_matrix.transpose()[user1]
        user2_ratings=user_movie_matrix.transpose()[user2]
        distance=hamming(user1_ratings, user2_ratings)
    except:
        distance=np.nan
    return distance
distance(100,200)

0.9987659399424106

In [18]:
# getting the N nearest neighbour based on the hamming distance of a user
user=1
def nearestNeighbours(user, K):
    allusers=pd.DataFrame(user_movie_matrix.index)
    # removing the active user
    allusers=allusers[allusers.userId!=user]
    allusers['distance']=allusers['userId'].apply(lambda x: distance(user, x))
    KNN=allusers.sort_values(['distance'],ascending=True)['userId'][:K]
    return KNN
KNN=nearestNeighbours(user,5)
KNN

413    414
379    380
44      45
596    597
447    448
Name: userId, dtype: int64

# Find Top N Recommendations

In [19]:
# Nearest neighbours ratings
NNR=user_movie_matrix[user_movie_matrix.index.isin(KNN)]
NNR

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
45,4.0,NaN,NaN,NaN,3.0,4.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,4.0,3.0,4.0,NaN,2.0,3.0,3.0,3.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,5.0,3.0,3.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,4.0,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Getting the average rating of each movie seen by Nearest Neighbours of active user
avgRating = NNR.apply(np.nanmean).dropna()
avgRating.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py:296: RuntimeWarning: Mean of empty slice
  values, self.f, axis=self.axis, dummy=dummy, labels=labels


movieId
1    4.400000
2    3.666667
3    3.500000
5    2.666667
6    3.750000
dtype: float64

In [21]:
# Removing the movies that is already been watched by user
movie_watched=user_movie_matrix.transpose()[user].dropna().index
movie_watched

Int64Index([   1,    3,    6,   47,   50,   70,  101,  110,  151,  157,
            ...
            3671, 3702, 3703, 3729, 3740, 3744, 3793, 3809, 4006, 5060],
           dtype='int64', name='movieId', length=232)

In [22]:
# Removing the movies which are already seen by user from avgRatings
avgRating = avgRating[~avgRating.index.isin(movie_watched)]
avgRating

movieId
2         3.666667
5         2.666667
7         2.333333
8         3.000000
10        3.750000
            ...   
184471    3.000000
184791    2.500000
187593    3.000000
187595    3.750000
188301    4.000000
Length: 3912, dtype: float64

In [26]:
# getting top N movies for a particular user based on nearest Neighbours
def TopN(user, N):
    KNN=nearestNeighbours(user,N)
    NNR=user_movie_matrix[user_movie_matrix.index.isin(KNN)]
    avgRating=NNR.apply(np.nanmean).dropna()
    movie_watched=user_movie_matrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(movie_watched)]
    topN=avgRating.sort_values(ascending=False).index[:N]
    movie=pd.DataFrame({'Movie':pd.Series(topN).apply(movieTitle), 'Genre':pd.Series(topN).apply(movieGenre)})
    return movie
    

In [24]:
favMovie(1,5)

,userId,movieId,rating,title,geners
231,1,5060,5.0,M*A*S*H (a.k.a. MASH) (1970),Comedy|Drama|War
185,1,2872,5.0,Excalibur (1981),Adventure|Fantasy
89,1,1291,5.0,Indiana Jones and the Last Crusade (1989),Action|Adventure
90,1,1298,5.0,Pink Floyd: The Wall (1982),Drama|Musical
190,1,2948,5.0,From Russia with Love (1963),Action|Adventure|Thriller


In [27]:
TopN(1,5)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py:296: RuntimeWarning: Mean of empty slice
  values, self.f, axis=self.axis, dummy=dummy, labels=labels


,Movie,Genre
0,"Batman: The Dark Knight Returns, Part 2 (2013)",Action|Animation
1,Nausicaä of the Valley of the Wind (Kaze no ta...,Adventure|Animation|Drama|Fantasy|Sci-Fi
2,Secretary (2002),Comedy|Drama|Romance
3,"Princess and the Warrior, The (Krieger und die...",Drama|Romance
4,True Grit (1969),Adventure|Drama|Western
